# Quantum Machine Learning

In this tutorial, we'll use a combination quantum-classical algorithm in order to train a model to predict data points in a curve. 

We start out by importing all of the necessary python packages, including PennyLane to build the quantum circuit, matplotlib to visualize our data, and Covalent to organize, track, and dispatch our workflow.

In [19]:
import pennylane as qml
from pennylane import numpy as np
#import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import covalent as ct

We'll start out by loading our data from a csv file. In this example, we use a gaussian curve, but any simple curve could be used.

In [20]:
@ct.electron
def load_data(filename):
    data = np.loadtxt(filename, delimiter=',')
    return data

In [21]:
data = load_data('gauss.txt')

As is, the dataset is too large! Quantum circuits function best with smaller datasets, so we define a new function to evenly select datapoints from our csv file. The below function, reduce_data, takes the dataset and number of data points n. We choose 7 data points, but we encourage the user to vary this number and see how it impacts the algorithm's performance.

In [22]:
@ct.electron
def reduce_data(data, n):
    interval = int(len(data)/n)
    return data[0::interval]

In [23]:
red_data = reduce_data(data, 10)
#plt.scatter(red_data[:,0], red_data[:,1])
#plt.plot(red_data[:,0], red_data[:,1], linestyle='--')
#plt.show()

The process of validation is an important part of machine learning. Validation involves separating your dataset into separate training and testing datasets. The training dataset is used to actually train the parameters of your model. Once your model has been sufficiently trained, its accuracy can be evaluated using the test dataset. The majority of the original dataset (usually 60-85%) is used for training data, while the remaining data (15-40%) is used for test data.

In the following step, we split our reduced dataset into training and test data, and separate the X and Y columns. There are a few ways to do this, but in this example we'll use scikit-learn's train_test_split() function. 

In [24]:
@ct.electron
def train_test_split(data, train_portion):
    from sklearn.model_selection import train_test_split
    X_train, X_test, Y_train, Y_test = train_test_split(data[:,0], data[:,1], train_size=train_portion, random_state=42)
    X_train = np.tensor(X_train)
    X_train.requires_grad = False
    return X_train, np.array(Y_train), np.tensor(X_test), np.array(Y_test)

In [25]:
X_train, Y_train, X_test, Y_test = train_test_split(red_data, 0.5)

#plt.plot(data[:,0], data[:,1], linestyle='--')
#plt.scatter(X_train, Y_train, label='Training Data')
#plt.scatter(X_test, Y_test, label='Test Data')
#plt.legend()
#plt.show()

In [26]:
@ct.electron
def plot_data(X_train, Y_train, X_test, Y_test):
    plt.scatter(X_train, Y_train, color='#219ebc', label='Training Data')
    plt.scatter(X_test, Y_test, color='#fb8500', alpha=0.5, label='Test Data')
    plt.legend()
    plt.savefig('input_data.png')

In [27]:
dev = qml.device('default.qubit', wires=1)

In [28]:
dev = qml.device('default.qubit', wires=1)
@ct.electron
def quantum_circuit(datapoint, params):
        datapoint = datapoint
        params = params
        @qml.qnode(dev)
        def qc():
                qml.RX(datapoint, wires=0)
                qml.Rot(params[0], params[1], params[2], wires=0)
                return qml.expval(qml.PauliZ(wires=0))
        expval = qc()
        return expval

In [29]:
@ct.electron
def cost_fn(params):
    #X, Y, X_test, Y_test = train_test_split(red_data, 0.5)
    predictions = make_predictions(X_train, params)
    total_losses = 0
    for i in range(len(Y_train)):
        loss = (predictions[i] - Y_train[i])**2
        total_losses += loss
    return total_losses
    #return cost

@ct.electron 
def update_parameters(opt, params, quantum_circuit):
    params = opt.step(cost_fn,params)
    return params


@ct.electron
def define_optimizer():
    opt = qml.AdamOptimizer(stepsize=0.01)
    return opt

In [30]:
@ct.electron
def make_predictions(X_test, params):
    test_predictions = []
    for x_test in X_test:
        prediction = quantum_circuit(x_test,params)
        test_predictions.append(prediction)
    return test_predictions

In [31]:
@ct.electron
def plot_results(X, Y, X_test, Y_test, predictions):
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax1.scatter(X, Y, c='#219ebc', label='Training data')
    ax1.scatter(X_test,Y_test, c='#fb8500', label='Test data')
    ax1.scatter(X_test,predictions, s=30, c='k', marker="x", label='Model Predicitons')
    plt.xlabel("Inputs")
    plt.ylabel("Outputs")
    plt.title("QML results")
    plt.legend(loc='upper right')
    plt.savefig('predictions.png')

In [32]:
'''
red_data = reduce_data(data=data, n=10)
X, Y, X_test, Y_test = train_test_split(red_data, 0.5)
@ct.lattice
@ct.electron
def optimize(X, opt):
    #opt = define_optimizer()
    params = np.array([0.01,0.01,0.01],requires_grad=True)
    for i in range (200):
        params = update_parameters(opt, params)
    return params

@ct.lattice
def workflow(X_train, Y_train, X_test, Y_test):
    X, Y, X_test, Y_test = train_test_split(red_data, 0.5)
    opt = define_optimizer()
    params = optimize(X=X, opt=opt)
    predictions = make_predictions(X_test=X_test, params=params)
    plot_results(X, Y, X_test, Y_test, predictions)
'''

'\nred_data = reduce_data(data=data, n=10)\nX, Y, X_test, Y_test = train_test_split(red_data, 0.5)\n@ct.lattice\n@ct.electron\ndef optimize(X, opt):\n    #opt = define_optimizer()\n    params = np.array([0.01,0.01,0.01],requires_grad=True)\n    for i in range (200):\n        params = update_parameters(opt, params)\n    return params\n\n@ct.lattice\ndef workflow(X_train, Y_train, X_test, Y_test):\n    X, Y, X_test, Y_test = train_test_split(red_data, 0.5)\n    opt = define_optimizer()\n    params = optimize(X=X, opt=opt)\n    predictions = make_predictions(X_test=X_test, params=params)\n    plot_results(X, Y, X_test, Y_test, predictions)\n'

In [33]:
@ct.lattice
@ct.electron
def optimize(X_train, Y_train, opt):
    Y_train = Y_train
    X_train = X_train
    params = np.array([0.01,0.01,0.01],requires_grad=True)
    for i in range (200):
        params = update_parameters(opt, params, quantum_circuit)
    return params


data = np.loadtxt('gauss.txt', delimiter=',')
@ct.lattice
def miniworkflow():
    red_data = reduce_data(data=data, n=10)
    X_train, Y_train, X_test, Y_test = train_test_split(data=red_data, train_portion=0.5)
    opt = define_optimizer()
    params = optimize(X_train=X_train, Y_train=Y_train, opt=opt)
    predictions = make_predictions(X_test=X_test, params=params)
    plot_results(X_train, Y_train, X_test, Y_test, predictions)


In [34]:
ct.dispatch(miniworkflow)()

/Users/annahughes/demonstration/covalent/covalent/_workflow/lattice.py:158: UserWarning: Please make sure you are not manipulating an object inside the lattice.
  warnings.warn(


NameError: name 'X' is not defined

In [ ]:
#params = workflow(X, Y, X_test, Y_test)
#print(params)

In [ ]:
#ct.dispatch(workflow)(X, Y, X_test, Y_test)